# Setup Ambiente

In [1]:
#!pip install firebase_admin pandas matplotlib seaborn --quiet

# Connessione a Firestore


In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import plotly.graph_objects as go
from prophet.plot import plot_plotly
import warnings

# Ignora i warning di pandas per una visualizzazione più pulita
warnings.filterwarnings('ignore')

In [3]:
# Inizializzazione Firebase Admin SDK
# Sostituisci il "path" con il percorso al tuo file di credenziali Firebase
cred = credentials.Certificate("path/to/your/.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
# Definizione device sotto analisi
device_id = "iliadbox-77F2A2"  # Sostituisci con l'ID del device che vuoi analizzare

# Estrazione Dati

## Prenotazioni

In [ ]:
prenotazioni_ref = db.collection('devices').document(device_id).collection('prenotazioni')
docs = prenotazioni_ref.stream()

data_list = []
for doc in docs:
    d = doc.to_dict()
    d['booking_id'] = doc.id
    # Conversione date
    if 'start_time' in d:
        d['start_time'] = pd.to_datetime(d['start_time'], utc=True, errors='coerce')
    if 'end_time' in d:
        d['end_time'] = pd.to_datetime(d['end_time'], utc=True, errors='coerce')
    data_list.append(d)

prenotazioni_df = pd.DataFrame(data_list)
if 'start_time' in prenotazioni_df and prenotazioni_df['start_time'].dtype == 'datetime64[ns, UTC]':
    prenotazioni_df['start_time'] = prenotazioni_df['start_time'].dt.tz_convert(None)
if 'end_time' in prenotazioni_df and prenotazioni_df['end_time'].dtype == 'datetime64[ns, UTC]':
    prenotazioni_df['end_time'] = prenotazioni_df['end_time'].dt.tz_convert(None)

prenotazioni_df.head()


## Analisi Prenotazioni

In [ ]:
# Calcolo della durata delle prenotazioni in ore
if 'start_time' in prenotazioni_df.columns and 'end_time' in prenotazioni_df.columns:
    prenotazioni_df['duration_hours'] = (prenotazioni_df['end_time'] - prenotazioni_df['start_time']) / pd.Timedelta(hours=1)
    durata_media = prenotazioni_df['duration_hours'].mean()
    print(f"Durata media delle prenotazioni: {durata_media:.2f} ore")

# Estrazione Access Log

In [ ]:
access_logs_ref = db.collection('devices').document(device_id).collection('access_logs')
docs = access_logs_ref.stream()

logs_data = []
for doc in docs:
    d = doc.to_dict()
    d['log_id'] = doc.id
    d['timestamp'] = pd.to_datetime(d['timestamp'], utc=True, errors='coerce')
    logs_data.append(d)

logs_df = pd.DataFrame(logs_data)
if 'timestamp' in logs_df and logs_df['timestamp'].dtype == 'datetime64[ns, UTC]':
    logs_df['timestamp'] = logs_df['timestamp'].dt.tz_convert(None)

logs_df.head()

## Analisi dei Logs

In [ ]:
logs_df['hour'] = logs_df['timestamp'].dt.hour
logs_df['date'] = logs_df['timestamp'].dt.date

# Frequenza azioni "porta aperta"
porta_aperta_df = logs_df[logs_df['action'] == 'porta aperta']
hourly = porta_aperta_df.groupby('hour').size().reset_index(name='count')

plt.figure(figsize=(10,5))
sns.lineplot(x='hour', y='count', data=hourly, marker='o')
plt.title(f"Frequenza Apertura Porta per Ora del Giorno - {device_id}")
plt.xticks(range(0,24))
plt.grid(True)
plt.show()

## Apertura porte giornaliere

In [ ]:
if 'timestamp' in logs_df.columns:
    logs_df['date'] = logs_df['timestamp'].dt.date
    aperture_df = logs_df[logs_df['action'] == 'porta aperta']
    daily_opens = aperture_df.groupby('date').size().reset_index(name='count')

    plt.figure(figsize=(14,7))
    ax = sns.barplot(x='date', y='count', data=daily_opens, color='green')
    plt.xticks(rotation=45)
    plt.title("Numero di Aperture Porta per Giorno", fontsize=16)
    plt.xlabel("Data", fontsize=14)
    plt.ylabel("Numero di Aperture", fontsize=14)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()

    for p in ax.patches:
        x = p.get_x() + p.get_width() / 2
        y = p.get_height()
        ax.text(x, y+0.1, str(int(y)), ha='center', va='bottom', fontsize=8, color='black')

    plt.show()


## HeatMap Aperture Per Giorno della Settimana

In [ ]:
if 'timestamp' in logs_df.columns:
    logs_df['day_of_week'] = logs_df['timestamp'].dt.day_name()
    logs_df['hour'] = logs_df['timestamp'].dt.hour
    opens = logs_df[logs_df['action'] == 'porta aperta']

    heatmap_data = opens.groupby(['day_of_week', 'hour']).size().unstack(fill_value=0)
    plt.figure(figsize=(12,6))
    sns.heatmap(heatmap_data, cmap='Blues', annot=True, fmt='d')
    plt.title("Frequenza Aperture per Giorno della Settimana e Ora (Singolo Device)", fontsize=16)
    plt.xlabel("Ora del Giorno", fontsize=14)
    plt.ylabel("Giorno della Settimana", fontsize=14)
    plt.tight_layout()
    plt.show()

## Numero di Allarmi per Giorno

In [ ]:
if 'action' in logs_df.columns:
    alarms_df = logs_df[logs_df['action'].str.contains("allarme_on|EFF", case=False, na=False)]
    if not alarms_df.empty:
        alarms_df['date'] = alarms_df['timestamp'].dt.date
        alarms_per_day = alarms_df.groupby('date').size().reset_index(name='count')

        plt.figure(figsize=(14,7))
        ax = sns.barplot(x='date', y='count', data=alarms_per_day, color='red')
        plt.xticks(rotation=45)
        plt.title("Numero di Allarmi Attivati per Giorno", fontsize=16)
        plt.xlabel("Data", fontsize=14)
        plt.ylabel("Numero di Allarmi", fontsize=14)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()

        for p in ax.patches:
            x = p.get_x() + p.get_width() / 2
            y = p.get_height()
            ax.text(x, y+0.1, str(int(y)), ha='center', va='bottom', fontsize=8, color='black')

        plt.show()
    else:
        print("Nessun allarme trovato nei log.")


# Uso di ML

## Clustering dei giorni (K-Means) su Aperture

In [ ]:
if 'timestamp' in logs_df.columns:
    logs_df['date'] = logs_df['timestamp'].dt.date

    # Conteggio aperture per giorno
    daily_opens = logs_df[logs_df['action'] == 'porta aperta'].groupby('date').size().reset_index(name='opens')

    if len(daily_opens) < 2:
        print("Non ci sono abbastanza dati di aperture per eseguire il clustering.")
    else:
        from sklearn.cluster import KMeans
        import matplotlib.pyplot as plt
        import seaborn as sns

        # Se hai pochi dati, prova con 2 cluster
        n_clusters = 2 if len(daily_opens) < 5 else 3

        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(daily_opens[['opens']])
        daily_opens['cluster'] = kmeans.labels_

        plt.figure(figsize=(14,7))
        sns.scatterplot(x='date', y='opens', data=daily_opens, hue='cluster', palette='Set2', s=100)
        plt.xticks(rotation=45)
        plt.title("Clustering dei Giorni per Numero di Aperture (Singolo Device)", fontsize=16)
        plt.xlabel("Data", fontsize=14)
        plt.ylabel("Numero di Aperture", fontsize=14)
        plt.legend(title='Cluster')
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()

        cluster_means = daily_opens.groupby('cluster')['opens'].mean()
        print("Media delle Aperture per Cluster:")
        print(cluster_means)


## Clustering (K-Means) su Allarmi 

In [ ]:
if 'action' in logs_df.columns:
    alarms_df = logs_df[logs_df['action'].str.contains("allarme_on|EFF", case=False, na=False)]
    if alarms_df.empty:
        print("Non ci sono allarmi per questo device.")
    else:
        alarms_df['date'] = alarms_df['timestamp'].dt.date
        daily_alarms = alarms_df.groupby('date').size().reset_index(name='alarms')

        if len(daily_alarms) < 2:
            print("Non ci sono abbastanza dati di allarmi per eseguire il clustering.")
        else:
            from sklearn.cluster import KMeans
            import matplotlib.pyplot as plt
            import seaborn as sns

            n_clusters = 2 if len(daily_alarms) < 5 else 3
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            kmeans.fit(daily_alarms[['alarms']])
            daily_alarms['cluster'] = kmeans.labels_

            plt.figure(figsize=(14,7))
            sns.scatterplot(x='date', y='alarms', data=daily_alarms, hue='cluster', palette='Set2', s=100)
            plt.xticks(rotation=45)
            plt.title("Clustering dei Giorni per Numero di Allarmi (Singolo Device)", fontsize=16)
            plt.xlabel("Data", fontsize=14)
            plt.ylabel("Numero di Allarmi", fontsize=14)
            plt.legend(title='Cluster')
            plt.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.show()

            cluster_means = daily_alarms.groupby('cluster')['alarms'].mean()
            print("Media degli Allarmi per Cluster:")
            print(cluster_means)


## Clusterizzare i giorni in base al numero di prenotazioni, aperture porta, effrazioni.

### Creiamo un DataFrame aggregato giornalmente

In [ ]:
logs_df['date'] = logs_df['timestamp'].dt.date
daily_stats = logs_df.groupby('date')['action'].value_counts().unstack(fill_value=0)
daily_stats.head()


### Normalizza i dati e usa KMeans per clusterizzare i giorni

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(daily_stats)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)

daily_stats['cluster'] = clusters
daily_stats.head()

# Osserva le medie per cluster
print(daily_stats.groupby('cluster').mean())
